# CreateInstance and DeleteInstance

<a href="#" onclick="history.back()">&lt;--- Back</a>

The creation of a CIM instance and in turn the creation of the underlying managed resource is achieved by calling the [`CreateInstance()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.CreateInstance) method. It takes a [`pywbem.CIMInstance`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.CIMInstance) object as input, which specifies the class and the initial properties for the CIM instance to be created, and returns a [`pywbem.CIMInstanceName`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.CIMInstanceName) object that references the new CIM instance.

The [`DeleteInstance()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.DeleteInstance) method takes a [`pywbem.CIMInstanceName`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.CIMInstanceName) object and deletes the referenced CIM instance and the represented managed resource, or rejects the operation if deletion is not supported.

For some CIM classes, it makes no sense to support creation or deletion of their CIM instances. For some others, that makes sense and is defined in their usage definitions in WBEM Management Profiles (see DMTF standard DSP1001). Often, management profiles that define a semantics for the creation or deletion of managed resources, leave that optional for an implementation to support. The implementation for a CIM class in the WBEM server (aka CIM provider) thus may or may not support creation or deletion of its instances and the represented managed resources.

Note that the `CIMInstance` object provided as input to `CreateInstance()` does not specfify an instance path (or if it does, it will be ignored). The determination of an instance path for the new CIM instance is completely left to the CIM provider in the WBEM server. For CIM classes with natural keys (key properties other than "InstanceID"), some CIM providers do honor initial values for some or all of the key properties provided in the input instance.

In [ ]:
from __future__ import print_function
import sys
import pywbem

username = 'user'
password = 'password'
classname = 'CIM_ComputerSystem'
namespace = 'root/interop'
server = 'http://localhost'

conn = pywbem.WBEMConnection(server, (username, password),
                             default_namespace=namespace,
                             no_verification=True)

filter_inst = pywbem.CIMInstance(
    'CIM_IndicationFilter',
    {'Name': 'pywbem_test',
     'Query': 'SELECT * FROM CIM_Indication',
     'QueryLanguage': 'WQL'})
    
print('Creating instance of class: %s' % filter_inst.classname)
try:
    filter_path = conn.CreateInstance(filter_inst, namespace)
except pywbem.Error as exc:
    if isinstance(exc, pywbem.CIMError) and \
       exc.status_code == pywbem.CIM_ERR_NOT_SUPPORTED:
        print('WBEM server does not support creation of dynamic filters.')
        filter_path = None
    else:
        print('CreateInstance failed: %s: %s' % (exc.__class__.__name__, exc))
        sys.exit(1)

if filter_path is not None:
    print('Created instance: %s' % filter_path)
    print('Deleting the instance again, to clean up')
    try:
        conn.DeleteInstance(filter_path)
    except pywbem.Error as exc:
        print('DeleteInstance failed: %s: %s' % (exc.__class__.__name__, exc))
        sys.exit(1)
    print('Deleted the instance')

This example has a somewhat more elaborated failure message that includes the type of exception that happened.

This example also shows how specific CIM errors can be detected: If creation of the CIM instance and the corresponding managed resource is not supported, this example code accepts that and does not error out. All other errors, including other CIM errors, cause an error exit.

PyWBEM maps CIM operation failures to the Python exception [`pywbem.CIMError`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.CIMError), and raises that in this case. The CIM status code is available as a numeric value in the `status_code` attribute of the exception object. See [CIM status codes](https://pywbem.readthedocs.io/en/latest/client.html#cim-status-codes) for a definition of the CIM status code values.

<a href="#" onclick="history.back()">&lt;--- Back</a>